In [18]:
import numpy as np
import os
import pandas as pd
import sys
import torch

from matplotlib import pyplot as plt
from tqdm import tqdm

project_dir = os.path.join(os.getcwd(),'..')
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [19]:
x = torch.arange(9).reshape(3,3)
idx = torch.tril_indices(3, 3, offset=-1)

# Usa los índices para obtener los elementos de la parte inferior triangular
lower_triangular_elements = x[idx[0], idx[1]]
x[idx[0], idx[1]] = -2
print(lower_triangular_elements)
print(x)

tensor([3, 6, 7])
tensor([[ 0,  1,  2],
        [-2,  4,  5],
        [-2, -2,  8]])


In [20]:
argwhere_format = torch.stack(idx, dim=1)
argwhere_format

TypeError: stack(): argument 'tensors' (position 1) must be tuple of Tensors, not Tensor

In [21]:
import torch

# Genera los índices de la parte inferior triangular de una matriz de 3x3
indices = torch.tril_indices(row=3, col=3, offset=0)

# Transponer y apilar los índices para obtener el formato similar a argwhere
argwhere_format = torch.stack((indices[0], indices[1]), dim=1)
print(argwhere_format)

tensor([[0, 0],
        [1, 0],
        [1, 1],
        [2, 0],
        [2, 1],
        [2, 2]])


In [22]:
x, y = zip(indices)
print(x, y)

(tensor([0, 1, 1, 2, 2, 2]),) (tensor([0, 0, 1, 0, 1, 2]),)


In [23]:
import torch

from VAE.nn.VariationalLayer import VariationalLayer
from torch.nn.parameter import Parameter
import torch.nn.functional as F
from torch import nn
import numpy as np

class AnomalyDetector(nn.Linear, VariationalLayer):
    '''
        This implementation is for anomaly detection.
    '''
    def __init__(self, in_features, out_features, bias=True, full_cov=False, sigma_anomaly=3) -> None:
        assert out_features > 1, "The Gaussian posterior variables must be greater than 1"

        super(AnomalyDetector, self).__init__(in_features, out_features, bias)
        self.log_sigma_weight = Parameter(torch.Tensor(out_features, in_features))
        self.log_sigma_bias = Parameter(torch.Tensor(out_features))

        if full_cov:
            # Correlation estimation
            self.corr_weight = Parameter(torch.Tensor(out_features, in_features))
            self.corr_bias = Parameter(torch.Tensor(out_features))
        
        torch.nn.init.xavier_uniform_(self.log_sigma_weight)
        self.log_sigma_bias.data.fill_(2*np.log((2*sigma_anomaly)))
        
        self.sigma_anomaly = sigma_anomaly
        self.mu, self.sigma = None, None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        mu = F.linear(x, self.weight, self.bias) 
        sigma = torch.exp(0.5 * F.linear(x, self.log_sigma_weight, self.log_sigma_bias))

        # if self.training:
        self.mu, self.sigma = mu, sigma

        # Reparameterization trick
        eps = torch.normal(0, torch.ones_like(sigma))
        return mu + sigma * eps

    def kl_reg(self, targets: torch.Tensor) -> torch.Tensor:
        # KL-Divergence regularization
        assert torch.all((targets == 0) | (targets == 1))
        sigma_2 = (torch.ones_like(targets) + ((self.sigma_anomaly-1) * targets)).unsqueeze(1).cuda()
        result = (torch.log(sigma_2) - torch.log(self.sigma) + (self.sigma**2 + self.mu**2)/(2*sigma_2**2) - 0.5)
        
        return result.mean() 

## Correlation and Covariance Implementation

In [24]:
import math
in_features, out_features = 10, 3
corr_size = math.comb(out_features, 2)
corr_weight = Parameter(torch.Tensor(corr_size, in_features))
corr_bias = Parameter(torch.Tensor(corr_size))

log_sigma_weight = Parameter(torch.Tensor(out_features, in_features))
log_sigma_bias = Parameter(torch.Tensor(out_features))

torch.nn.init.xavier_uniform_(log_sigma_weight)
log_sigma_bias.data.fill_(2*np.log((2*1)))

torch.nn.init.xavier_uniform_(corr_weight)
corr_bias.data.fill_(0)

x = torch.rand((5, 10))

In [25]:
corr = F.tanh(F.linear(x, corr_weight, corr_bias))
sigma = torch.exp(0.5 * F.linear(x, log_sigma_weight, log_sigma_bias))

print(corr)
print(sigma)

tensor([[-0.4742, -0.7323,  0.2421],
        [-0.6136, -0.6057,  0.0196],
        [-0.6130, -0.4806, -0.1536],
        [-0.5227, -0.8324,  0.7438],
        [-0.4280, -0.4303,  0.5668]], grad_fn=<TanhBackward0>)
tensor([[2.2982, 2.5568, 1.8826],
        [2.0268, 2.8106, 1.5734],
        [2.0550, 2.6878, 1.4660],
        [3.1454, 2.7031, 2.1273],
        [2.1495, 4.4167, 1.7010]], grad_fn=<ExpBackward0>)


In [26]:
corr.shape

torch.Size([5, 3])

In [27]:
cov1 = corr[:, 0] * (sigma[:, 0] * sigma[:, 1])
cov2 = corr[:, 1] * (sigma[:, 0] * sigma[:, 2])
cov3 = corr[:, 2] * (sigma[:, 1] * sigma[:, 2])
cov1, cov2, cov3

(tensor([-2.7865, -3.4956, -3.3859, -4.4437, -4.0632], grad_fn=<MulBackward0>),
 tensor([-3.1683, -1.9317, -1.4478, -5.5695, -1.5735], grad_fn=<MulBackward0>),
 tensor([ 1.1655,  0.0869, -0.6052,  4.2771,  4.2585], grad_fn=<MulBackward0>))

In [28]:
idx = torch.tril_indices(corr.size(1), corr.size(1), offset=-1)

a = torch.zeros((1,3,3))
a[:, *tuple(idx)] = 1
a

tensor([[[0., 0., 0.],
         [1., 0., 0.],
         [1., 1., 0.]]])

In [29]:
print(*tuple(idx))

tensor([1, 2, 2]) tensor([0, 0, 1])


In [30]:
idx

tensor([[1, 2, 2],
        [0, 0, 1]])

In [31]:
# idx = list(combinations(range(out_features), 2))
idx = torch.tril_indices(corr.size(1), corr.size(1), offset=-1)
tuple(idx)

(tensor([1, 2, 2]), tensor([0, 0, 1]))

In [32]:
corr.size()

torch.Size([5, 3])

In [33]:
corr_matrix = torch.zeros((corr.size(0), out_features, out_features))
corr_matrix[:, *tuple(idx)] = corr
corr_matrix

tensor([[[ 0.0000,  0.0000,  0.0000],
         [-0.4742,  0.0000,  0.0000],
         [-0.7323,  0.2421,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [-0.6136,  0.0000,  0.0000],
         [-0.6057,  0.0196,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [-0.6130,  0.0000,  0.0000],
         [-0.4806, -0.1536,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [-0.5227,  0.0000,  0.0000],
         [-0.8324,  0.7438,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000],
         [-0.4280,  0.0000,  0.0000],
         [-0.4303,  0.5668,  0.0000]]], grad_fn=<CopySlices>)

In [34]:
def covariance_matrix(corr, sigma):
    idx = torch.tril_indices(corr.size(1), corr.size(1), offset=-1)
    cov_matrix = torch.zeros((corr.size(0), corr.size(1), corr.size(1)))

    test = sigma[:, idx.T]

    cov_matrix[:, *tuple(idx)] = corr * test.prod(dim=2)
    cov_matrix = cov_matrix + torch.diag_embed(sigma**2)
    return cov_matrix

In [35]:
covariance_matrix(corr, sigma)

tensor([[[ 5.2818,  0.0000,  0.0000],
         [-2.7865,  6.5371,  0.0000],
         [-3.1683,  1.1655,  3.5442]],

        [[ 4.1080,  0.0000,  0.0000],
         [-3.4956,  7.8993,  0.0000],
         [-1.9317,  0.0869,  2.4755]],

        [[ 4.2232,  0.0000,  0.0000],
         [-3.3859,  7.2243,  0.0000],
         [-1.4478, -0.6052,  2.1492]],

        [[ 9.8935,  0.0000,  0.0000],
         [-4.4437,  7.3066,  0.0000],
         [-5.5695,  4.2771,  4.5253]],

        [[ 4.6204,  0.0000,  0.0000],
         [-4.0632, 19.5074,  0.0000],
         [-1.5735,  4.2585,  2.8933]]], grad_fn=<AddBackward0>)

In [36]:
print(corr)

tensor([[-0.4742, -0.7323,  0.2421],
        [-0.6136, -0.6057,  0.0196],
        [-0.6130, -0.4806, -0.1536],
        [-0.5227, -0.8324,  0.7438],
        [-0.4280, -0.4303,  0.5668]], grad_fn=<TanhBackward0>)


In [37]:
idx = torch.tril_indices(corr.size(1), corr.size(1), offset=-1)
test = sigma[:, idx.T]

cov = test.prod(dim=-1) * corr

corr.shape, test.shape, cov.shape

(torch.Size([5, 3]), torch.Size([5, 3, 2]), torch.Size([5, 3]))

In [38]:
test, test.prod(dim=-1)

(tensor([[[2.5568, 2.2982],
          [1.8826, 2.2982],
          [1.8826, 2.5568]],
 
         [[2.8106, 2.0268],
          [1.5734, 2.0268],
          [1.5734, 2.8106]],
 
         [[2.6878, 2.0550],
          [1.4660, 2.0550],
          [1.4660, 2.6878]],
 
         [[2.7031, 3.1454],
          [2.1273, 3.1454],
          [2.1273, 2.7031]],
 
         [[4.4167, 2.1495],
          [1.7010, 2.1495],
          [1.7010, 4.4167]]], grad_fn=<IndexBackward0>),
 tensor([[5.8760, 4.3266, 4.8134],
         [5.6965, 3.1890, 4.4221],
         [5.5235, 3.0127, 3.9403],
         [8.5022, 6.6911, 5.7501],
         [9.4938, 3.6562, 7.5127]], grad_fn=<ProdBackward1>))

In [39]:
2.4587 * 1.7624, 1.5704 * 1.7624, 1.5704 * 2.4587

(4.33321288, 2.76767296, 3.86114248)

In [40]:
# idx = torch.triu_indices(3, 3, offset=1)
idx = torch.tril_indices(3, 3, offset=-1)
test = sigma[:, idx.T]
test

tensor([[[2.5568, 2.2982],
         [1.8826, 2.2982],
         [1.8826, 2.5568]],

        [[2.8106, 2.0268],
         [1.5734, 2.0268],
         [1.5734, 2.8106]],

        [[2.6878, 2.0550],
         [1.4660, 2.0550],
         [1.4660, 2.6878]],

        [[2.7031, 3.1454],
         [2.1273, 3.1454],
         [2.1273, 2.7031]],

        [[4.4167, 2.1495],
         [1.7010, 2.1495],
         [1.7010, 4.4167]]], grad_fn=<IndexBackward0>)

In [41]:
sigma[0, [[0,1], [0,2], [1,2]]]

tensor([[2.2982, 2.5568],
        [2.2982, 1.8826],
        [2.5568, 1.8826]], grad_fn=<IndexBackward0>)

In [248]:
def correlation_tests():
    # sigma = torch.ones(1,5)*2
    sigma = torch.arange(1, 6).reshape(1, 5)
    corr = torch.tensor([[1, .5, .25, .2, .1, .09, .08, .05, .02, .01]])

    idx = torch.tril_indices(5, 5, offset=-1)

    print(tuple(idx))
    corr_matrix = torch.zeros((1, 5, 5))
    corr_matrix[:, *tuple(idx)] = corr
    print(corr_matrix.shape, torch.diag_embed(sigma).shape)
    print((corr_matrix + torch.eye(*corr_matrix.shape[1:])))


    cov_matrix = torch.zeros((1, 5, 5))
    # cov_10 = corr_matrix[:, 1, 0] * (sigma[0, 1] * sigma[0, 0])
    # cov_20 = corr_matrix[:, 2, 0] * (sigma[0, 2] * sigma[0, 0])
    # cov_21 = corr_matrix[:, 2, 1] * (sigma[0, 2] * sigma[0, 1])
    # cov_30 = corr_matrix[:, 3, 0] * (sigma[0, 3] * sigma[0, 0])
    # cov_31 = corr_matrix[:, 3, 1] * (sigma[0, 3] * sigma[0, 1])
    # cov_32 = corr_matrix[:, 3, 2] * (sigma[0, 3] * sigma[0, 2])

    for i in range(1, 5):
        for j in range(i):
            cov_matrix[:, i, j] = corr_matrix[:, i, j] * (sigma[0, i] * sigma[0, j])

    cov_matrix = cov_matrix + torch.diag_embed(sigma**2)
    print(cov_matrix)
    
    return sigma, corr, cov_matrix

def correlation_tests(n_samples=1, n_variables=3):
    '''
        Inneficent way to generate the correlation matrix. This function
        is only for testing purposes.
    '''
    sigma = torch.arange(1, (n_samples*n_variables)+1).reshape(n_samples, n_variables)
    
    n_corr = math.comb(n_variables, 2)
    corr = torch.linspace(-1, 1, steps=n_samples*n_corr).reshape(n_samples, n_corr)
    idx = torch.tril_indices(n_variables, n_variables, offset=-1)
    corr_matrix = torch.zeros((n_samples, n_variables, n_variables))
    for n in range(n_samples):
        corr_matrix[n, *tuple(idx)] = corr[n]

    cov_matrix = torch.zeros((n_samples, n_variables, n_variables))

    for n in range(n_samples):
        for i in range(1, n_variables):
            for j in range(i):
                corr_matrix[n, i, j]
                cov_matrix[n, i, j] = corr_matrix[n, i, j] * (sigma[n, i] * sigma[n, j])

    cov_matrix = cov_matrix + torch.diag_embed(sigma)
        
    return sigma, corr, cov_matrix

sigma, corr, cov_matrix = correlation_tests(n_samples=3, n_variables=5)


cov_matrix

tensor([[[  1.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [ -2.0000,   2.0000,   0.0000,   0.0000,   0.0000],
         [ -2.7931,  -5.1724,   3.0000,   0.0000,   0.0000],
         [ -3.1724,  -5.7931,  -7.8621,   4.0000,   0.0000],
         [ -2.9310,  -5.1724,  -6.7241,  -7.5862,   5.0000]],

        [[  6.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [-13.0345,   7.0000,   0.0000,   0.0000,   0.0000],
         [-11.5862,  -9.6552,   8.0000,   0.0000,   0.0000],
         [ -5.5862,  -2.1724,   2.4828,   9.0000,   0.0000],
         [  6.2069,  12.0690,  19.3103,  27.9310,  10.0000]],

        [[ 11.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [ 50.0690,  12.0000,   0.0000,   0.0000,   0.0000],
         [ 64.1034,  80.6897,  13.0000,   0.0000,   0.0000],
         [ 90.2759, 110.0690, 131.7931,  14.0000,   0.0000],
         [130.8621, 155.1724, 181.5517, 210.0000,  15.0000]]])

In [262]:
torch.bmm(cov_matrix, cov_matrix.transpose(1, 2))

tensor([[[ 1.0000e+00, -2.0000e+00, -2.7931e+00, -3.1724e+00, -2.9310e+00],
         [-2.0000e+00,  8.0000e+00, -4.7586e+00, -5.2414e+00, -4.4828e+00],
         [-2.7931e+00, -4.7586e+00,  4.3555e+01,  1.5239e+01,  1.4768e+01],
         [-3.1724e+00, -5.2414e+00,  1.5239e+01,  1.2144e+02,  6.1784e+01],
         [-2.9310e+00, -4.4828e+00,  1.4768e+01,  6.1784e+01,  1.6311e+02]],

        [[ 3.6000e+01, -7.8207e+01, -6.9517e+01, -3.3517e+01,  3.7241e+01],
         [-7.8207e+01,  2.1890e+02,  8.3434e+01,  5.7606e+01,  3.5791e+00],
         [-6.9517e+01,  8.3434e+01,  2.9146e+02,  1.0556e+02, -3.3960e+01],
         [-3.3517e+01,  5.7606e+01,  1.0556e+02,  1.2309e+02,  2.3843e+02],
         [ 3.7241e+01,  3.5791e+00, -3.3960e+01,  2.3843e+02,  1.4372e+03]],

        [[ 1.2100e+02,  5.5076e+02,  7.0514e+02,  9.9303e+02,  1.4395e+03],
         [ 5.5076e+02,  2.6509e+03,  4.1779e+03,  5.8408e+03,  8.4142e+03],
         [ 7.0514e+02,  4.1779e+03,  1.0789e+04,  1.6382e+04,  2.3270e+04],
        

In [261]:
cov_matrix.transpose(1, 2)[0]

tensor([[ 1.0000, -2.0000, -2.7931, -3.1724, -2.9310],
        [ 0.0000,  2.0000, -5.1724, -5.7931, -5.1724],
        [ 0.0000,  0.0000,  3.0000, -7.8621, -6.7241],
        [ 0.0000,  0.0000,  0.0000,  4.0000, -7.5862],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  5.0000]])

In [260]:
cov_matrix[0]

tensor([[ 1.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-2.0000,  2.0000,  0.0000,  0.0000,  0.0000],
        [-2.7931, -5.1724,  3.0000,  0.0000,  0.0000],
        [-3.1724, -5.7931, -7.8621,  4.0000,  0.0000],
        [-2.9310, -5.1724, -6.7241, -7.5862,  5.0000]])

In [44]:
corr, sigma

(tensor([[-1.0000, -0.9310, -0.8621, -0.7931, -0.7241, -0.6552, -0.5862, -0.5172,
          -0.4483, -0.3793],
         [-0.3103, -0.2414, -0.1724, -0.1034, -0.0345,  0.0345,  0.1034,  0.1724,
           0.2414,  0.3103],
         [ 0.3793,  0.4483,  0.5172,  0.5862,  0.6552,  0.7241,  0.7931,  0.8621,
           0.9310,  1.0000]]),
 tensor([[ 1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10],
         [11, 12, 13, 14, 15]]))

In [45]:
def covariance_matrix(corr, sigma):
    n_variables = sigma.size(1)
    idx = torch.tril_indices(n_variables, n_variables, offset=-1)
    cov_matrix = torch.zeros((corr.size(0), n_variables, n_variables))
    test = sigma[:, idx.T]

    cov_matrix[:, *tuple(idx)] = corr * test.prod(dim=2)
    cov_matrix = cov_matrix + torch.diag_embed(sigma**2)
    return cov_matrix

In [46]:
cov_matrix_opt = covariance_matrix(corr, sigma)

torch.all(cov_matrix == cov_matrix_opt)

tensor(True)

In [49]:
corr

tensor([[-1.0000, -0.9310, -0.8621, -0.7931, -0.7241, -0.6552, -0.5862, -0.5172,
         -0.4483, -0.3793],
        [-0.3103, -0.2414, -0.1724, -0.1034, -0.0345,  0.0345,  0.1034,  0.1724,
          0.2414,  0.3103],
        [ 0.3793,  0.4483,  0.5172,  0.5862,  0.6552,  0.7241,  0.7931,  0.8621,
          0.9310,  1.0000]])

In [48]:
cov_matrix

tensor([[[  1.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [ -2.0000,   4.0000,   0.0000,   0.0000,   0.0000],
         [ -2.7931,  -5.1724,   9.0000,   0.0000,   0.0000],
         [ -3.1724,  -5.7931,  -7.8621,  16.0000,   0.0000],
         [ -2.9310,  -5.1724,  -6.7241,  -7.5862,  25.0000]],

        [[ 36.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [-13.0345,  49.0000,   0.0000,   0.0000,   0.0000],
         [-11.5862,  -9.6552,  64.0000,   0.0000,   0.0000],
         [ -5.5862,  -2.1724,   2.4828,  81.0000,   0.0000],
         [  6.2069,  12.0690,  19.3103,  27.9310, 100.0000]],

        [[121.0000,   0.0000,   0.0000,   0.0000,   0.0000],
         [ 50.0690, 144.0000,   0.0000,   0.0000,   0.0000],
         [ 64.1034,  80.6897, 169.0000,   0.0000,   0.0000],
         [ 90.2759, 110.0690, 131.7931, 196.0000,   0.0000],
         [130.8621, 155.1724, 181.5517, 210.0000, 225.0000]]])

In [51]:
0.6552 * math.sqrt(144*196)

110.0736

### ---

In [331]:
import math
in_features, out_features = 10, 3
corr_size = math.comb(out_features, 2)
corr_weight = Parameter(torch.Tensor(corr_size, in_features))
corr_bias = Parameter(torch.Tensor(corr_size))

log_sigma_weight = Parameter(torch.Tensor(out_features, in_features))
log_sigma_bias = Parameter(torch.Tensor(out_features))

torch.nn.init.xavier_uniform_(log_sigma_weight)
log_sigma_bias.data.fill_(2*np.log((2*1)))

torch.nn.init.xavier_uniform_(corr_weight)
corr_bias.data.fill_(0)

x = torch.rand((5, 10))

In [464]:
corr = F.tanh(F.linear(x, corr_weight, corr_bias))
sigma = torch.exp(0.5 * F.linear(x, log_sigma_weight, log_sigma_bias))

print(corr)
print(sigma**2)

tensor([[-0.2903,  0.1116,  0.5963],
        [-0.5898,  0.2591,  0.7299],
        [ 0.1199,  0.3307,  0.7757],
        [-0.4145,  0.5578,  0.8469],
        [-0.1558, -0.2250,  0.2757]], grad_fn=<TanhBackward0>)
tensor([[3.3669, 4.7113, 1.2586],
        [4.6382, 1.8477, 1.4570],
        [3.2549, 3.9438, 0.9478],
        [2.2858, 3.6933, 0.8907],
        [2.9648, 4.2810, 1.7595]], grad_fn=<PowBackward0>)


In [471]:
def covariance_matrix(corr, sigma):
    n_variables = sigma.size(1)
    idx = torch.tril_indices(n_variables, n_variables, offset=-1)
    cov_matrix = torch.zeros((corr.size(0), n_variables, n_variables))
    test = sigma[:, idx.T]

    cov_matrix[:, *tuple(idx)] = corr * test.prod(dim=2)
    cov_matrix = cov_matrix + cov_matrix.transpose(-1, -2) + torch.diag_embed(sigma**2)
    # cov_matrix = cov_matrix + torch.diag_embed(sigma)
    return cov_matrix

cov_matrix = covariance_matrix(corr, sigma)
cov_matrix

tensor([[[ 3.3669, -1.1562,  0.2297],
         [-1.1562,  4.7113,  1.4521],
         [ 0.2297,  1.4521,  1.2586]],

        [[ 4.6382, -1.7267,  0.6736],
         [-1.7267,  1.8477,  1.1976],
         [ 0.6736,  1.1976,  1.4570]],

        [[ 3.2549,  0.4294,  0.5809],
         [ 0.4294,  3.9438,  1.4997],
         [ 0.5809,  1.4997,  0.9478]],

        [[ 2.2858, -1.2042,  0.7960],
         [-1.2042,  3.6933,  1.5362],
         [ 0.7960,  1.5362,  0.8907]],

        [[ 2.9648, -0.5551, -0.5139],
         [-0.5551,  4.2810,  0.7568],
         [-0.5139,  0.7568,  1.7595]]], grad_fn=<AddBackward0>)

In [ ]:
tensor([[[ 1.8349,  0.0000,  0.0000],
         [-1.1562,  2.1706,  0.0000],
         [ 0.2297,  1.4521,  1.1219]],

        [[ 2.1536,  0.0000,  0.0000],
         [-1.7267,  1.3593,  0.0000],
         [ 0.6736,  1.1976,  1.2071]],

        [[ 1.8041,  0.0000,  0.0000],
         [ 0.4294,  1.9859,  0.0000],
         [ 0.5809,  1.4997,  0.9736]],

        [[ 1.5119,  0.0000,  0.0000],
         [-1.2042,  1.9218,  0.0000],
         [ 0.7960,  1.5362,  0.9438]],

        [[ 1.7218,  0.0000,  0.0000],
         [-0.5551,  2.0691,  0.0000],
         [-0.5139,  0.7568,  1.3265]]]

20

### Test nuevo

In [390]:
A = torch.tensor([[1,0,0,0],[-4,2,0,0], [5,6,3,0], [7,8,9,4]])
Cov = A @ A.T
sigma = torch.diag(Cov).sqrt()
print(Cov)
print(sigma)


tensor([[  1,  -4,   5,   7],
        [ -4,  20,  -8, -12],
        [  5,  -8,  70, 110],
        [  7, -12, 110, 210]])
tensor([ 1.0000,  4.4721,  8.3666, 14.4914])


In [391]:
math.sqrt(20), math.sqrt(70)

(4.47213595499958, 8.366600265340756)

In [401]:
math.sqrt(7**2 - 12)

6.082762530298219

In [397]:
var = sigma**2
print(var)
idx = torch.tril_indices(4, 4, offset=-1)
L = torch.zeros_like(A, dtype=torch.float)
L[*tuple(idx)] = A[*tuple(idx)].float()

print(L)

diag_indices = torch.arange(4)
L[diag_indices, diag_indices] = torch.sqrt((sigma**2 - (L**2).sum(dim=1))).float()
L

tensor([  1.0000,  20.0000,  70.0000, 210.0000])
tensor([[ 0.,  0.,  0.,  0.],
        [-4.,  0.,  0.,  0.],
        [ 5.,  6.,  0.,  0.],
        [ 7.,  8.,  9.,  0.]])


tensor([[ 1.0000,  0.0000,  0.0000,  0.0000],
        [-4.0000,  2.0000,  0.0000,  0.0000],
        [ 5.0000,  6.0000,  3.0000,  0.0000],
        [ 7.0000,  8.0000,  9.0000,  4.0000]])

In [418]:
# Covarianze trinagular lower matrix
C_l = torch.zeros_like(Cov, dtype=torch.float)
idx_ = torch.tril_indices(4, 4, offset=0)
C_l[*tuple(idx_)] = Cov[*tuple(idx_)].float()

# Cholesky decomposition
L_l = L

C_l, L_l

(tensor([[  1.,   0.,   0.,   0.],
         [ -4.,  20.,   0.,   0.],
         [  5.,  -8.,  70.,   0.],
         [  7., -12., 110., 210.]]),
 tensor([[ 1.0000,  0.0000,  0.0000,  0.0000],
         [-4.0000,  2.0000,  0.0000,  0.0000],
         [ 5.0000,  6.0000,  3.0000,  0.0000],
         [ 7.0000,  8.0000,  9.0000,  4.0000]]))

In [437]:
import torch

# Ejemplo de matriz de covarianza triangular inferior C_l
# C_l = torch.tensor([[4.0, 0.0, 0.0, 0.0],
#                     [2.0, 5.0, 0.0, 0.0],
#                     [1.0, 3.0, 6.0, 0.0],
#                     [0.5, 1.5, 2.5, 7.0]])

# Inicializar L_l con ceros
L_l = torch.zeros_like(C_l)
print(C_l)
# Calcular L_l manualmente
for i in range(C_l.size(0)):
    for j in range(i + 1):
        # print(i,j)
        if i == j:
            L_l[i, j] = torch.sqrt(C_l[i, j] - torch.sum(L_l[i, :j] ** 2))
        else:
            L_l[i, j] = (C_l[i, j] - torch.sum(L_l[i, :j] * L_l[j, :j])) / L_l[j, j]

# Mostrar L_l
print("Matriz triangular inferior L_l:")
print(L_l)

tensor([[  1.,   0.,   0.,   0.],
        [ -4.,  20.,   0.,   0.],
        [  5.,  -8.,  70.,   0.],
        [  7., -12., 110., 210.]])
Matriz triangular inferior L_l:
tensor([[ 1.,  0.,  0.,  0.],
        [-4.,  2.,  0.,  0.],
        [ 5.,  6.,  3.,  0.],
        [ 7.,  8.,  9.,  4.]])


In [436]:
(110 - (8*3 + 7*3)) / 3

21.666666666666668

In [449]:
print(L_l[3,  :2])
print(L_l[2, :2])
print(L_l[3,  :2] * L_l[2, :2])
print(110 - torch.sum(L_l[3,  :2] * L_l[2, :2]))
print((110 - torch.sum(L_l[3,  :2] * L_l[2, :2]))/ L_l[2, 2])

tensor([7., 8.])
tensor([5., 6.])
tensor([35., 48.])
tensor(27.)
tensor(9.)


In [450]:
(-8 - (5*-4)) / 2

6.0

In [373]:
L[diag_indices, diag_indices].float()

tensor([0., 0., 0.])

In [366]:
 torch.tensor([1,1,1]).shape

torch.Size([3])

In [357]:
L[diag_indices, diag_indices]

tensor([0, 0, 0])

In [356]:
diag_indices

tensor([0, 1, 2])

In [355]:
sigma**2 - (L**2).sum(dim=1)

tensor([1.0000, 4.0000, 9.0000])

In [ ]:
idx = torch.tril_indices(3, 3, offset=-1)
L = torch.zeros_like(Cov)
L[*tuple(idx)] = Cov[*tuple(idx)]



print(L)
(L**2)

tensor([[ 0,  0,  0],
        [ 4,  0,  0],
        [ 5, 32,  0]])


tensor([[   0,    0,    0],
        [  16,    0,    0],
        [  25, 1024,    0]])

In [455]:
torch.exp(torch.tensor(-100))




tensor(3.7835e-44)